In [1]:
import json
import os
import random
import fnmatch
import openpyxl
from datetime import datetime

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import re

In [2]:
sns.set_style("darkgrid")

In [3]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [4]:
def clean_str(s):
    d = {
        "’": "'",
        "\xa0": " ",
        "/":","
        }
    for x in d:
        s = s.replace(x, d[x]).strip()
    return s

In [5]:
month_dict = {
    "January" : "Janvier",
"February": "Février",
"March": "Mars",
"April" : "Avril",
"May" : "Mai",
"June" : "Juin",
"July" : "Juillet",
"August" : "Août",
"September":"Septembre",
"October":"Octobre",
"November":"Novembre",
"December":"Décembre"}

In [6]:
taxo_dep_df = pd.read_csv('refs/taxo_deps.csv', dtype={'dep':str, 'reg':str})
taxo_dep_df['dep'] = taxo_dep_df['dep'].apply(lambda x: x.zfill(2))
taxo_dep_df['reg'] = taxo_dep_df['reg'].apply(lambda x: x.zfill(2))
dep_list = list(taxo_dep_df['dep'].unique())


In [7]:
taxo_reg_df = pd.read_csv('refs/taxo_regions.csv', dtype={'reg':str})
taxo_reg_df['reg'] = taxo_reg_df['reg'].apply(lambda x: x.zfill(2))
reg_list = list(taxo_reg_df['reg'].unique())


In [8]:
data_dir_path = './data/'

In [9]:
propilot_path = os.path.join("data")

In [10]:
df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [11]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

File loaded :  dim_activity.csv
File loaded :  dim_tree_nodes.csv
File loaded :  dim_top_levels.csv
File loaded :  dim_maturities.csv
File loaded :  dim_period_pp_ch.csv
File loaded :  dim_snapshots.csv
File loaded :  dim_effects.csv
File loaded :  dim_properties.csv
File loaded :  dim_states_pp_ch.csv
File loaded :  dim_structures.csv
File loaded :  fact_financials_enr_pp_ch.csv


/home/mquentin-adc/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:

Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.



File loaded :  fact_property_values.csv


In [12]:
def format_date(raw_date):
    # Convertie un format (20210131, 2020) -> (2021-01-31, 2020)
    str_date = str(raw_date)
    if str_date == 'nan':
        return raw_date
    
    if type(str_date) == int:
        print(str_date)
    if len(str_date) == 8:
        return str_date[:4] + '-' + str_date[4:6] + '-' + str_date[6:]
    else:
        return str_date
    

df_dict['fact_financials']['period_id'] = df_dict['fact_financials']['period_id'].apply(lambda x: format_date(x))
df_dict['dim_period']['period_id'] = df_dict['dim_period']['period_id'].apply(lambda x: format_date(x))

In [13]:
# Retirer les lignes ayant un period quarter year commençant par 11 -> nonsense
df_dict['dim_period'] = df_dict['dim_period'][~df_dict['dim_period']['period_quarter_year'].str.startswith('11', na=False)]

In [14]:
df = (df_dict["fact_financials"]
      .merge(df_dict["dim_tree_nodes"], left_on="tree_node_id", right_on="tree_node_id") 
      .merge(df_dict["dim_effects"], left_on="effect_id", right_on="effect_id") 
      .merge(df_dict["dim_states"], left_on="state_id", right_on="state_id")  
      .merge(df_dict["dim_period"], left_on="period_id", right_on="period_id", how = 'left')
      .merge(df_dict["dim_structures"], left_on="structure_id", right_on="structure_id"))

In [15]:
def check_data_merge(facts, df):
    num_pp_recs = facts[facts['financials_source'] == 'proPilot'].shape[0]
    num_df_recs = df.shape[0]
    assert num_pp_recs == num_df_recs, f"Le nombre d'enregistrements proPilot diffèrent : avant {num_pp_recs} - après {num_df_recs}"
    

check_data_merge(df_dict["fact_financials"], df)

In [16]:
def extract_dep_code(expr):
    nums = re.findall(r'D\d+', expr)
    if expr.endswith('D2A'):
        return '2A'
    elif expr.endswith('D2B'):
        return '2B'
    elif expr.endswith('E00'):
        return '00'
    return nums[0][1:].zfill(2) if len(nums) > 0 else None


df['dep_code'] = df['tree_node_code'].apply(lambda x: extract_dep_code(x))

In [17]:
def check_cols(cols, df):
    assert set(cols).issubset(df.columns), f"Colonnes manquantes : {set(cols) - set(df.columns)}"

In [18]:
cols = ["tree_node_name", "structure_name", "effect_id", "state_id", "period_date", "period_month_tri", "period_month_year", "financials_cumulated_amount", "dep_code"]
check_cols(cols, df)

df=df[cols]
df.rename(columns={"period_month_year":"Date", "financials_cumulated_amount":"valeur"}, inplace=True)

In [19]:
forbidden_period_value = ["Y", "Total"]

In [20]:
def clean_mesure_name(tree_node_name):
    raw_mesure = tree_node_name.split('/')[1].strip() if '/' in tree_node_name else tree_node_name
    # nettoyage de la colonne mesure, on enlève un point surnuméraire.
    mesure = re.sub('\.', "", raw_mesure)
    mesure = clean_str(mesure)
    return mesure

In [21]:
def check_single_sum_dep_equals_nat(df_copy, mesure, indic):
    nat_values = df_copy[(df_copy['structure_name'] == 'Mesure') & 
                    (df_copy['mesure'] == mesure) &
                    (df_copy['effect_id'] == indic)][['period_date', 'valeur']].set_index('period_date').to_dict()['valeur']

    dep_indic_values = df_copy[(df_copy['structure_name'] == 'Département') & 
                            (df_copy['mesure'] == mesure) & 
                            (df_copy['effect_id'] == indic)].groupby('period_date').sum().to_dict()['valeur']

    for date in dep_indic_values.keys():
        if date not in nat_values:
            #print(f"-- National pas de {date}")
            continue
        dep_val = round(dep_indic_values[date], 2)
        nat_val = round(nat_values[date], 2)
        #assert dep_val == nat_val, f"Somme départementale : {dep_val} - Valeur récupérée nationale : {nat_val}\n{date} - {mesure} - {indic}"
        # TODO : subtituer le if-else par un assert quand on aura la confirmation que les données sont cohérentes
        if dep_val == nat_val:
            pass
            #print(f'ok - {mesure} - {indic} - {date}')
        else:
            print(f'KO - {mesure} - {indic} - {date} | {dep_val} - {nat_val}')
            

def check_sum_dep_equals_nat(df):
    """ Ce test a été rajouté après avoir constaté des différences entre somme des départements et valeurs nationales."""
    df_copy = df.copy()
    df_copy = df_copy.loc[
                (~df_copy.period_month_tri.isin(forbidden_period_value)) &
                (df_copy.state_id == 'Valeur Actuelle') &
                (~df_copy.valeur.isna())].copy()

    df_copy["mesure"] = df_copy["tree_node_name"].apply(lambda x: clean_mesure_name(x))

    mesures = df_copy['mesure'].unique()
    for mesure in mesures:
        indics = df_copy[df_copy['mesure'] == mesure]['effect_id'].unique()
        for indic in indics:
            check_single_sum_dep_equals_nat(df_copy, mesure, indic)

            
check_sum_dep_equals_nat(df)

KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2020-12-31T00:00:00.0000000 | 0.0 - 121520.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-03-31T00:00:00.0000000 | 0.0 - 801520.0
KO - Hydrogène – Ecosystèmes territoriaux - Nombre de projets soutenus - ETH2 - 2020-12-31T00:00:00.0000000 | 0.0 - 7.0
KO - Hydrogène – Ecosystèmes territoriaux - Nombre de projets soutenus - ETH2 - 2021-03-31T00:00:00.0000000 | 0.0 - 21.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-05-31T00:00:00.0000000 | 5429.0 - 6371.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-06-30T00:00:00.0000000 | 7024.0 - 11048.0
KO - Restauration écologique - Nombre de projets soutenus - REA1 - 2021-04-30T00:00:00.0000000 | 157.0 - 182.0
KO - Restauration écologique - Nombre de projets soutenus - REA1 

In [22]:
df.rename(columns={"effect_id":"indicateur"}, inplace=True)
df.indicateur = df.indicateur.str.strip()
df["short_indic"] = df.indicateur.apply(lambda x: x.split("-")[0].strip())
df["indic_id"] = df.indicateur.apply(lambda x: x.split("-")[1].strip())

In [23]:
df = df.loc[(~df.period_month_tri.isin(forbidden_period_value)) &
    (df.state_id == 'Valeur Actuelle') &
    (~df.valeur.isna())].copy()

In [24]:
# Filtrer les lignes ayant une date ultérieure à la date d'aujourd'hui
today = datetime.today().strftime('%Y-%m-%d')

date_series = df['period_date'].apply(lambda x: x.split('T')[0])
df['format_date'] = pd.to_datetime(date_series)

df = df[df['format_date'] <= today]
df = df.drop('format_date', axis=1)

df["departement"] = df["tree_node_name"].apply(lambda x: x.split("/")[0].strip())
df["mesure"] = df["tree_node_name"].apply(lambda x: x.split("/")[-1].strip())
df.drop(columns=["tree_node_name"], inplace=True)

# nettoyage de la colonne mesure, on enlève un point surnuméraire.
df["mesure"].replace("\.", "", regex=True, inplace=True)
df.mesure = df.mesure.apply(lambda x: clean_str(x))

# traduit les mois dans la colonne date
df.Date = df.Date.replace(month_dict, regex=True)

# enrichit avec les noms de département/région
df = df.merge(taxo_dep_df[["dep", "reg", "libelle"]], 
                          how="left", left_on="dep_code", right_on="dep")\
                   .merge(taxo_reg_df[["reg", "nccenr"]], how="left", left_on="reg", right_on="reg")

df.rename(columns={"nccenr":"region"}, inplace=True)
df.drop(['dep_code'], axis=1, inplace=True)  # Supprime code dep utilisé pour la jointure

In [25]:
df_dep = df.loc[df.structure_name == "Département"].copy()
df_dep.drop(columns=["structure_name"], inplace=True)

dep_indic = set(df_dep.indicateur.unique())
df_reg = df.loc[(df.structure_name == "Région") 
       & (~df.indicateur.isin(dep_indic))].copy()
df_reg.drop(columns=["structure_name"], inplace=True)

reg_indic = set(df_reg.indicateur.unique())
df_nat = df.loc[(df.structure_name == "Mesure") 
       & (~df.indicateur.isin(reg_indic))
       & (~df.indicateur.isin(dep_indic))].copy()
df_nat.drop(columns=["structure_name"], inplace=True)

In [26]:
df_dep["short_indic"] = df_dep.indicateur.apply(lambda x: x.split("-")[0].strip())
df_dep.short_indic = df_dep.short_indic.apply(lambda x: clean_str(x))

In [27]:
def check_single_sum_dep_equals_nat_2(df_copy, df_nat, mesure, indic):
    nat_values = df_nat[(df_nat['mesure'] == mesure) &
                    (df_nat["indicateur"] == indic)][['period_date', 'valeur']].set_index('period_date').to_dict()['valeur']

    dep_indic_values = df_copy[(df_copy['mesure'] == mesure) & 
                                (df_copy['indicateur'] == indic)].groupby('period_date').sum().to_dict()['valeur']

    for date in dep_indic_values.keys():
        if date not in nat_values:
            #print(f"-- National pas de {date}")
            continue
        dep_val = round(dep_indic_values[date], 2)
        nat_val = round(nat_values[date], 2)
        #assert dep_val == nat_val, f"Somme départementale : {dep_val} - Valeur récupérée nationale : {nat_val}\n{date} - {mesure} - {indic}"
        # TODO : subtituer le if-else par un assert quand on aura la confirmation que les données sont cohérentes
        if dep_val == nat_val:
            pass
            #print(f'ok - {mesure} - {indic} - {date}')
        else:
            print(f'KO - {mesure} - {indic} - {date} | {dep_val} - {nat_val}')


def check_sum_dep_equals_nat_2(df_dep, df_nat):
    mesures = df_dep['mesure'].unique()
    for mesure in mesures:
        indics = df_dep[df_dep['mesure'] == mesure]['indicateur'].unique()
        for indic in indics:
            check_single_sum_dep_equals_nat_2(df_dep, df_nat, mesure, indic)
            
            
check_sum_dep_equals_nat_2(df_dep, df)

KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2020-12-31T00:00:00.0000000 | 0.0 - 121520.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-03-31T00:00:00.0000000 | 0.0 - 801520.0
KO - Hydrogène – Ecosystèmes territoriaux - Nombre de projets soutenus - ETH2 - 2020-12-31T00:00:00.0000000 | 0.0 - 7.0
KO - Hydrogène – Ecosystèmes territoriaux - Nombre de projets soutenus - ETH2 - 2021-03-31T00:00:00.0000000 | 0.0 - 21.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-05-31T00:00:00.0000000 | 5429.0 - 6371.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-06-30T00:00:00.0000000 | 7024.0 - 11048.0
KO - Restauration écologique - Nombre de projets soutenus - REA1 - 2021-04-30T00:00:00.0000000 | 157.0 - 182.0
KO - Restauration écologique - Nombre de projets soutenus - REA1 

In [28]:
dict_indicateur = {'Nombre de repas servis dans les restaurants universitaires au tarif à 1€' : "Nombre de repas servis",
 'Montant cumulé de l’investissement total ainsi déclenché' : "Montant cumumé de l'investissement total",
 'Nombre d’entreprises bénéficiaires du dispositif' : "Nombre d'entreprises bénéficiaires",
 'Nombre de bâtiments Etat dont le marché de rénovation est notifié' : 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental' : 'Nombre de projets',
 'Nombre d\'exploitations certifiées "haute valeur environnementale"' : 'Nombre d’exploitations certifiées',
 'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements' : 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables' : 'Nombre de bonus octroyés à des véhicules électriques',
 "Quantité de matières plastiques évitées ou dont le recyclage ou l'intégration a été soutenue" : 'Quantité de matières plastiques évitées',
 #'Montant total des travaux associés aux dossiers validés' : 'Montant total des travaux',
 'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle)' : 'Nombre de nouveaux projets',
 'Montant de l’investissement total déclenché' : 'Montant de l’investissement total',
 'Nombre de projets de tourisme durable financés' : 'Nombre de projets de tourisme durable financés',
 'Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés' : 'Nombre de projets de rénovation',
 'Montant total investi pour la rénovation de monuments historiques appartenant aux collectivités territoriales' :'Montant total investi pour la rénovation',
 'Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés' : 'Nombre de projets de rénovation',
 "Nombre de contrats de professionnalisation bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats de professionnalisation',
 "Nombre de contrats d'apprentissage bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats d’apprentissage',
 "Nombre d'aides à l'embauche des travailleurs handicapés": "Nombre d'aides à l'embauche des travailleurs handicapés",
 'Nombre de projets locaux soutenus  (rénovation, extension, création de lignes' :'Nombre de projets locaux soutenus',
 #'Nombre de dossiers MaPrimeRénov validés': 'Nombre de dossiers MaPrimeRénov bénéficiaires ("particulier" et "copropriété")',
 #'Nombre de dossiers MaPrimeRénov bénéficiaires ("particulier" et "copropriété")': 'Nombre de bénéficiaires',
 "Nombre de dossiers MaPrimeRénov' payés": "Nombre de dossiers payés",
  "Nombre d'entreprises bénéficiares": "Nombre d'entreprises bénéficiaires"
                  }

In [29]:
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

In [30]:
dict_mesures  = {
 "Appels à projets dédiés à l'efficacité énergétique et à l'évolution des procédés en faveur de la décarbonation de l'industrie": "AAP Efficacité énergétique",
 "CIE jeunes" : "Contrats Initiatives Emploi (CIE) Jeunes",
 'France Num' : 'France Num : aide à la numérisation des TPE,PME,ETI',
 'Guichet efficacité énergétique dans industrie' : 'Guichet efficacité énergétique',
 "Modernisation des filières automobiles et aéronautiques" : "Modernisation des filières automobiles et aéronautiques",
 "PEC jeunes": "Parcours emploi compétences (PEC) Jeunes",
 'Relocalisation : soutien aux projets industriels dans les territoires' : 'AAP Industrie : Soutien aux projets industriels territoires',
 'Relocalisation : sécurisation des approvisionnements critiques' : 'AAP Industrie : Sécurisation approvisionnements critiques',
 'Renforcement des subventions de Business France (chèque export, chèque VIE)' : 'Renforcement subventions Business France',
 "Soutien à la modernisation industrielle et renforcement des compétences dans la filière nucléaire" : "AAP industrie : modernisation industrielle et renforcement des compétences dans la filière nucléaire",
 'Soutien à la recherche aéronautique civil' :'Soutien recherche aéronautique civil',
 'Rénovation bâtiments Etats' : 'Rénovation des bâtiments Etats (marchés notifiés)'
}

In [31]:
df_dep["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)

In [32]:
def get_df_sum_indicator(df_dep, indicators_to_sum, new_indicator, new_indic, new_mesure):
    df_dep = df_dep.copy()
    df_temp = df_dep.loc[df_dep.indicateur.str.contains(indicators_to_sum, regex=True)].groupby(["Date", "dep"]).sum().copy()
    df_temp["indicateur"] = new_indicator
    df_temp["mesure"] = new_mesure
    df_temp = (df_temp
            .merge(df_dep.drop(columns=["indicateur", "indic_id", "short_indic", "mesure", "short_mesure", "valeur"])
                   .drop_duplicates(["Date", "dep"]), 
             on=["Date", "dep"]))
    df_temp["short_indic"] = new_indicator + " - " + new_indic
    df_temp["short_mesure"] = new_mesure
    df_temp["indic_id"] = new_indic
    #df_temp.fillna("NaN", inplace=True)
    return df_temp

In [33]:
df_temp = get_df_sum_indicator(
    df_dep, 
    "PEE3|EEI1|CBC3",
    "Nombre d’entreprises ayant reçu l’aide",
    "SSS1",
    "Décarbonation de l'industrie (Appel à projets EE + Guichet EE + Chaleur bas carbone)")
df_temp2 = get_df_sum_indicator(
    df_dep, 
    "PEE2|CBC2",
    "Montant cumulé de l’investissement total ainsi déclenché",
    "SSS2",
    "Décarbonation de l'industrie (Appel à projets EE + Guichet EE + Chaleur bas carbone)")

In [34]:
df_dep = pd.concat([df_dep, df_temp, df_temp2])

In [35]:
def check_dep_coherence(df):
    assert df['dep'].isnull().sum() == 0, "Certaines lignes ne possèdent pas de code département."
    
    deps_test = sorted(df['dep'].unique())
    deps_true = sorted(taxo_dep_df['dep'].unique())
    assert deps_test == deps_true, "Départements ne concordent pas."
    
    assert sorted(df['libelle'].unique()) == sorted(taxo_dep_df['libelle'])
    

def check_reg_coherence(df):
    assert df['reg'].isnull().sum() == 0, "Certaines lignes ne possèdent pas de code région."
    
    regs_test = sorted(df['reg'].unique())
    regs_true = sorted(taxo_reg_df['reg'].unique())
    assert regs_test == regs_true, "Régions ne concordent pas."
        
    
def check_df_dep_enr_format(df_dep_enr):
    assert df_dep_enr.isnull().sum().sum() == 0
    assert df_dep_enr.shape[0] == df_dep.shape[0]

In [36]:
check_dep_coherence(df_dep)
check_reg_coherence(df_dep)
check_df_dep_enr_format(df_dep)

In [40]:
df_dep

,indicateur,state_id,period_date,period_month_tri,Date,valeur,short_indic,indic_id,departement,mesure,dep,reg,libelle,region,short_mesure
0,Nombre de locaux commerciaux connectables à la...,Valeur Actuelle,2020-12-31T00:00:00.0000000,Dec,Décembre 2020,190200.0,Nombre de locaux commerciaux connectables à la...,THD1,Sarthe,Généralisation de la fibre optique,72,52,Sarthe,Pays de la Loire,Généralisation de la fibre optique
1,Nombre de locaux commerciaux connectables à la...,Valeur Actuelle,2020-12-31T00:00:00.0000000,Dec,Décembre 2020,31000.0,Nombre de locaux commerciaux connectables à la...,THD1,Ardèche,Généralisation de la fibre optique,07,84,Ardèche,Auvergne-Rhône-Alpes,Généralisation de la fibre optique
2,Nombre de locaux commerciaux connectables à la...,Valeur Actuelle,2020-12-31T00:00:00.0000000,Dec,Décembre 2020,26000.0,Nombre de locaux commerciaux connectables à la...,THD1,Lozère,Généralisation de la fibre optique,48,76,Lozère,Occitanie,Généralisation de la fibre optique
3,Nombre de locaux commerciaux connectables à la...,Valeur Actuelle,2020-12-31T00:00:00.0000000,Dec,Décembre 2020,176000.0,Nombre de locaux commerciaux connectables à la...,THD1,Pyrénées-Orientales,Généralisation de la fibre optique,66,76,Pyrénées-Orientales,Occitanie,Généralisation de la fibre optique
4,Nombre de locaux commerciaux connectables à la...,Valeur Actuelle,2020-12-31T00:00:00.0000000,Dec,Décembre 2020,54100.0,Nombre de locaux commerciaux connectables à la...,THD1,Corse-du-Sud,Généralisation de la fibre optique,2A,94,Corse-du-Sud,Corse,Généralisation de la fibre optique
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,Montant cumulé de l’investissement total ainsi...,Valeur Actuelle,2020-11-30T00:00:00.0000000,Nov,Novembre 2020,0.0,Montant cumulé de l’investissement total ainsi...,SSS2,Terres australes et antarctiques françaises,Décarbonation de l'industrie (Appel à projets ...,984,984,Terres australes et antarctiques françaises,Terres australes et antarctiques françaises,Décarbonation de l'industrie (Appel à projets ...
977,Montant cumulé de l’investissement total ainsi...,Valeur Actuelle,2020-11-30T00:00:00.0000000,Nov,Novembre 2020,0.0,Montant cumulé de l’investissement total ainsi...,SSS2,Wallis-et-Futuna,Décarbonation de l'industrie (Appel à projets ...,986,COM,Wallis-et-Futuna,Collectivités d’outre-mer,Décarbonation de l'industrie (Appel à projets ...
978,Montant cumulé de l’investissement total ainsi...,Valeur Actuelle,2020-11-30T00:00:00.0000000,Nov,Novembre 2020,0.0,Montant cumulé de l’investissement total ainsi...,SSS2,Polynésie française,Décarbonation de l'industrie (Appel à projets ...,987,COM,Polynésie française,Collectivités d’outre-mer,Décarbonation de l'industrie (Appel à projets ...
979,Montant cumulé de l’investissement total ainsi...,Valeur Actuelle,2020-11-30T00:00:00.0000000,Nov,Novembre 2020,0.0,Montant cumulé de l’investissement total ainsi...,SSS2,Nouvelle-Calédonie,Décarbonation de l'industrie (Appel à projets ...,988,988,Nouvelle-Calédonie,Nouvelle-Calédonie,Décarbonation de l'industrie (Appel à projets ...


In [38]:
df_nat.head()

,indicateur,state_id,period_date,period_month_tri,Date,valeur,short_indic,indic_id,departement,mesure,dep,reg,libelle,region
26293,Nombre de projets soutenus - FRE1,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Nombre de projets soutenus,FRE1,Frêt,Frêt,NaN,NaN,NaN,NaN
26294,Nombre de projets soutenus - FRE1,Valeur Actuelle,2021-02-28T00:00:00.0000000,Feb,Février 2021,0.0,Nombre de projets soutenus,FRE1,Frêt,Frêt,NaN,NaN,NaN,NaN
26295,Nombre de projets soutenus - FRE1,Valeur Actuelle,2021-03-31T00:00:00.0000000,Mar,Mars 2021,0.0,Nombre de projets soutenus,FRE1,Frêt,Frêt,NaN,NaN,NaN,NaN
26296,Nombre de projets soutenus - FRE1,Valeur Actuelle,2021-04-30T00:00:00.0000000,Apr,Avril 2021,0.0,Nombre de projets soutenus,FRE1,Frêt,Frêt,NaN,NaN,NaN,NaN
26297,Nombre de projets soutenus - FRE1,Valeur Actuelle,2021-05-31T00:00:00.0000000,May,Mai 2021,0.0,Nombre de projets soutenus,FRE1,Frêt,Frêt,NaN,NaN,NaN,NaN


In [39]:
df_dep.to_csv("pp_dep.csv", sep=";")